In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1456, -0.1177,  0.0783, -0.0644,  0.0577, -0.0711,  0.2576,  0.1433,
         -0.1062,  0.0483],
        [-0.2327, -0.0538,  0.2181, -0.0833, -0.0372, -0.0508,  0.1952,  0.0945,
         -0.0379,  0.0123]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 1.4687e-02,  1.0486e-01, -5.9995e-02,  6.1786e-02, -6.4957e-02,
          2.8403e-01, -6.5001e-02, -2.5954e-02, -5.1439e-02,  4.0994e-01],
        [-1.0944e-03,  2.4289e-06,  3.4374e-02, -5.7390e-02, -5.3010e-02,
          3.1192e-01,  1.5578e-01,  1.2082e-01, -4.2883e-02,  5.4527e-01]],
       grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0201, -0.2048, -0.0801,  0.0982, -0.0835, -0.3391,  0.3833, -0.1631,
         -0.0298,  0.2886],
        [ 0.0232, -0.2545, -0.1353,  0.0816, -0.0420, -0.2455,  0.4485, -0.0866,
         -0.1425,  0.2474]], grad_fn=<AddmmBackward0>)

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(0.1280, grad_fn=<SumBackward0>)

In [8]:
X.shape

torch.Size([2, 20])